In [1]:
import requests
import uuid
import json
import pandas as pd
import datetime


# utility functions

# get token from bird api
def get_auth_token(guid):
    url = 'https://api.birdapp.com/user/login'
    headers = {
        'User-Agent': 'Bird/4.41.0 (co.bird.Ride; build:37; iOS 12.3.1) Alamofire/4.41.0',
        'Device-id': guid,
        'Platform': 'ios',
        'App-Version': '4.41.0',
        'Content-Type': 'application/json',
    }
    payload = {
        'email': '{}@example.com'.format(guid)
    }
    r = requests.post(url=url, data=json.dumps(payload), headers=headers)
    token = r.json().get('token')
    return token


# get nearby scooters use the token above
def get_nearby_scooters(token, lat, long):
    url = 'https://api.birdapp.com/bird/nearby'
    params = {
        'latitude': lat,
        'longitude': long,
        'radius': 1000
    }
    headers = {
        'Authorization': 'Bird {}'.format(token),
        'Device-id': '{}'.format(guid),
        'App-Version': '4.41.0',
        'Location': json.dumps({
            'latitude': lat,
            'longitude': long,
            'altitude': 500,
            'accuracy': 100,
            'speed': -1,
            'heading': -1
        })
    }
    r = requests.get(url=url, params=params, headers=headers)

    scooters = r.json().get('birds')

    return scooters


# split location dictionary into latitude and longitude
def split_location(df, location_label='location'):
    locs = df[location_label].apply(pd.Series)
    df_new = pd.concat([df, locs], axis=1)
    df_new = df_new.drop(location_label, axis=1)
    return df_new


# token extract
guid = str(uuid.uuid1())
token = get_auth_token(guid)
assert token is not None

# store city info
cities = [{'name': 'Mel_Rose', 'location': (34.0407, -118.2468)},
          {'name': 'DTLA', 'location': (34.0837, -118.3550)},
          {'name': 'Santa_Monica', 'location': (34.0195, -118.4912)}]
cities_df = pd.DataFrame(cities)

# extract scooters info for each city
for index, city in cities_df.iterrows():
    lat = city['location'][0]
    long = city['location'][1]
    birds_json = get_nearby_scooters(token, lat, long)
    birds_df = pd.DataFrame(birds_json)
    split_location(birds_df)
    path = 'data/{}_{}.csv'.format(city['name'], datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
    birds_df.to_csv(path)


In [ ]:
   bj = get_nearby_scooters(token, lat, long)
    bdf = pd.DataFrame(bj)
    split_location(bdf)
    